
# Лекция 5. Контекстные менеджеры в Python
## Почему контекстные менеджеры — это важно?

Всякий раз, когда мы работаем с ресурсами, такими как файлы, соединения с базами данных или сетевые соединения, мы сталкиваемся с важной задачей — корректно управлять этими ресурсами, особенно при возникновении ошибок. Контекстные менеджеры позволяют нам делать это очень элегантно и безопасно.

Контекстные менеджеры дают нам возможность автоматически управлять ресурсами, гарантируя их правильное открытие и закрытие. Вместо того чтобы вручную заботиться о закрытии файлов или освобождении соединений, мы можем использовать конструкцию with, которая сама позаботится обо всем. Давайте разберемся, как это работает.

### Основы работы с with

Когда мы используем `with`, Python автоматически вызывает методы `__enter__` и `__exit__` для объектов, которые поддерживают протокол контекстного менеджера. Это позволяет нам, например, открыть файл, работать с ним, а затем автоматически закрыть, даже если в процессе возникнет ошибка.

Пример использования контекстного менеджера с файлом:

In [ ]:
with open('example.txt', 'w') as file:
    file.write('Hello, World!')

Здесь происходит следующее:

В момент начала работы с блоком with вызывается метод `__enter__`, который открывает файл.
После выполнения блока (или при возникновении ошибки) вызывается метод `__exit__`, который закрывает файл.
Реализация контекстного менеджера: методы `__enter_`_ и `__exit__`

Теперь давайте создадим свой собственный контекстный менеджер. Для этого нам нужно реализовать два метода: `__enter__` и `__exit__`.

- `__enter__` выполняет необходимую подготовку перед выполнением кода внутри блока with.
- `__exit__` выполняет очистку и освобождение ресурсов после выполнения блока.
Пример контекстного менеджера для работы с базой данных:

In [ ]:
class DatabaseConnection:
    def __enter__(self):
        print("Подключаемся к базе данных...")
        self.connection = "Соединение с БД"
        return self.connection

    def __exit__(self, exc_type, exc_val, exc_tb):
        print("Закрываем соединение с базой данных...")
        self.connection = None

# Использование контекстного менеджера
with DatabaseConnection() as db:
    print(f"Работаем с: {db}")


В этом примере:

- Метод `__enter__` открывает соединение с базой данных.
- Метод `__exit__` закрывает это соединение, даже если во время работы с базой данных произошла ошибка.

Пример управления ресурсами

Контекстные менеджеры особенно полезны при работе с ресурсами, такими как файлы или сетевые соединения. Давайте рассмотрим пример использования контекстного менеджера для работы с файлами и базами данных.

Пример работы с файлом:

In [ ]:
class FileHandler:
    def __enter__(self):
        print("Открываем файл...")
        self.file = open('data.txt', 'w')
        return self.file

    def __exit__(self, exc_type, exc_val, exc_tb):
        print("Закрываем файл...")
        self.file.close()

# Использование контекстного менеджера для работы с файлом
with FileHandler() as file:
    file.write("Работаем с файлом...")


Здесь мы открываем файл с помощью `with`, и файл автоматически закрывается по завершению блока кода.

### Заключение

Контекстные менеджеры — это мощный инструмент для эффективного управления ресурсами. Они помогают нам избежать утечек памяти и других ресурсов, гарантируя, что все операции выполняются правильно, даже если возникают ошибки. Использование контекстных менеджеров улучшает код, делая его более читаемым и надежным.

## Слоты и управление памятью
**Почему важно управлять памятью?**

В Python, как и в любом другом языке программирования, управление памятью — это не только вопрос оптимизации, но и вопрос стабильности приложения. В стандартных классах Python все атрибуты объекта хранятся в словаре `__dict__`, что позволяет добавлять новые атрибуты динамически. Однако этот подход требует дополнительных ресурсов, особенно если объекты создаются в большом количестве.

Иногда нам нужно более эффективно управлять памятью. В таких случаях на помощь приходит использование атрибута `__slots__`.

## Что такое __slots__?

Когда мы используем `__slots__`, Python не создает для объекта обычный словарь `__dict__` для хранения атрибутов. Вместо этого создается специальная структура данных, которая ограничивает набор атрибутов, и использует более эффективную память для их хранения.

Пример использования `__slots__`:

In [ ]:
class Car:
    __slots__ = ['make', 'model']

    def __init__(self, make, model):
        self.make = make
        self.model = model

# Создаем объект
car = Car("Toyota", "Camry")
car.make = "Honda"  # Это сработает
car.model = "Civic"  # Это сработает

# Попытка добавить новый атрибут вызовет ошибку
# car.color = "Red"  # AttributeError: 'Car' object has no attribute 'color'


Здесь мы ограничили объект Car только атрибутами make и model. Попытка создать другие атрибуты приведет к ошибке.

### Экономия памяти с `__slots__`

Предположим, у нас есть класс с тысячами объектов. Каждый объект будет занимать меньше памяти, если мы используем `__slots__`, поскольку Python не создает для каждого объекта словарь `__dict__`.

Пример:

In [ ]:
import sys

class Car:
    __slots__ = ['make', 'model']

    def __init__(self, make, model):
        self.make = make
        self.model = model

car1 = Car("Toyota", "Camry")
car2 = Car("Honda", "Civic")

# Сравним использование памяти
print(sys.getsizeof(car1))  # Использует меньше памяти
print(sys.getsizeof(car2))  # Использует меньше памяти


### Когда использовать `__slots__`?

Использование `__slots__` полезно в случае:

Когда у вас много однотипных объектов, например, если вы работаете с большими данными.
Когда нужно оптимизировать использование памяти, чтобы улучшить производительность.
Однако стоит помнить, что использование `__slots__` ограничивает гибкость, так как мы не можем добавлять новые атрибуты динамически.

### Заключение

`__slots__` — это удобный инструмент для экономии памяти, особенно когда необходимо создавать множество объектов с фиксированным набором атрибутов. Использование `__slots__` позволяет значительно снизить затраты на память и повысить производительность, но требует более строгого подхода к проектированию классов.

## Сложные иерархии наследования

**Наследование** — это один из столпов объектно-ориентированного программирования. Однако, с увеличением сложности системы, иерархии классов могут становиться всё более запутанными. Проектирование таких систем требует особого внимания, чтобы избежать дублирования кода и сложностей при расширении системы.

Пример сложной иерархии наследования

Предположим, у нас есть несколько классов для разных видов транспорта. Создадим простую иерархию классов:

In [ ]:
class Vehicle:
    def start_engine(self):
        print("Двигатель запущен!")

class Car(Vehicle):
    def drive(self):
        print("Машина едет!")

class ElectricCar(Car):
    def charge(self):
        print("Заряжая электромобиль...")

class GasCar(Car):
    def refuel(self):
        print("Заправляем бензином...")


Здесь у нас есть базовый класс `Vehicle`, от которого наследуются `Car`, а затем `ElectricCar` и `GasCar`. Мы разделяем классы на электромобили и автомобили с бензиновым двигателем.

### Управление зависимостями и расширяемость

Когда системы становятся сложными, важно управлять зависимостями между классами. Если один класс зависит от другого, нужно чётко понимать, как эти зависимости будут влиять на расширение системы.

Пример:

In [ ]:
class HybridCar(ElectricCar, GasCar):
    def __init__(self):
        print("Создаем гибридный автомобиль!")

# Создаем объект гибридного автомобиля
hybrid = HybridCar()
hybrid.charge()  # Работает от ElectricCar
hybrid.refuel()  # Работает от GasCar


В этом примере `HybridCar` наследует два класса — `ElectricCar` и `GasCar`, и теперь может использовать оба их метода. Это демонстрирует, как несколько наследуемых классов могут быть комбинированы, чтобы создать новый функционал.

### Метод разрешения порядка (MRO)

Когда классы имеют сложные иерархии, важно понимать, какой метод будет вызван при наследовании от нескольких классов. В Python для этого используется алгоритм разрешения порядка методов (MRO).

In [ ]:
print(HybridCar.__mro__)


MRO определяет порядок, в котором Python будет искать методы в иерархии классов. Это позволяет нам контролировать, какой класс будет "первым" при поиске методов или атрибутов.

### Заключение

Сложные иерархии наследования позволяют создавать гибкие и расширяемые системы. Однако важно помнить о принципах управления зависимостями и следить за порядком наследования, чтобы избежать проблем с производительностью или непредсказуемым поведением.